# 🎵 이벤트 추천 시스템 - 완전한 모델 학습 데모

이 노트북은 다중 모델 기반 이벤트 추천 시스템의 전체 학습 과정을 보여줍니다.

## 📋 목차
1. 환경 설정 및 데이터 로드
2. 데이터 전처리 및 필터링
3. TF-IDF 모델 학습
4. LSA 모델 학습
5. Word2Vec 대안 모델 학습
6. 하이브리드 모델 구축
7. 모델 성능 비교
8. 추천 결과 시연

## 1. 환경 설정 및 데이터 로드

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

# 머신러닝 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
import re

# 플롯 설정
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style('whitegrid')

print("🚀 모든 라이브러리가 성공적으로 로드되었습니다!")

In [ ]:
# 기존 모델에서 데이터 로드
model_path = Path('../model/recommender_ko.joblib')

if model_path.exists():
    base_model = joblib.load(model_path)
    df_original = base_model['df']
    meta_preprocessor = base_model['pre']
    print(f"✅ 원본 데이터 로드 완료: {len(df_original):,}개 이벤트")
    print(f"📈 데이터셋 크기: {df_original.shape}")
    print(f"📋 컬럼: {list(df_original.columns)}")
else:
    print("❌ 모델 파일을 찾을 수 없습니다. 먼저 백엔드를 실행해주세요.")
    df_original = None

## 2. 데이터 전처리 및 필터링

In [ ]:
# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트 전처리 함수"""
    if pd.isna(text):
        return ''
    text = re.sub(r'[^\w\s]', ' ', str(text).lower())
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def filter_valid_events(df):
    """유효한 이벤트 데이터만 필터링"""
    print("🔧 데이터 필터링 및 전처리 중...")
    print(f"원본 데이터: {len(df):,}개 이벤트")
    
    # 기본 전처리
    df_filtered = df.copy()
    df_filtered['content_clean'] = df_filtered['content'].apply(preprocess_text)
    df_filtered['place_clean'] = df_filtered['place'].apply(preprocess_text)
    df_filtered['location_clean'] = df_filtered['loc_sigu'].apply(preprocess_text)
    
    # 필터링 조건
    # 1. content가 비어있지 않고 10자 이상
    valid_content = (df_filtered['content'].notna()) & (df_filtered['content'].str.len() >= 10)
    print(f"유효한 내용(10자 이상): {valid_content.sum():,}개")
    
    # 2. place 정보가 있는 것
    valid_place = df_filtered['place'].notna() & (df_filtered['place'].str.len() > 0)
    print(f"유효한 장소 정보: {valid_place.sum():,}개")
    
    # 3. 가격 정보가 있는 것 (0보다 큰 값)
    valid_price = df_filtered['price_adv'].notna() & (df_filtered['price_adv'] > 0)
    print(f"유효한 가격 정보: {valid_price.sum():,}개")
    
    # 모든 조건을 만족하는 데이터만 선택
    valid_mask = valid_content & valid_place & valid_price
    df_filtered = df_filtered[valid_mask].reset_index(drop=True)
    
    print(f"✅ 필터링 완료: {len(df_filtered):,}개 이벤트 (제거된 것: {len(df) - len(df_filtered):,}개)")
    print(f"📊 필터링 비율: {len(df_filtered)/len(df)*100:.1f}%")
    
    return df_filtered

if df_original is not None:
    # 데이터 필터링 적용
    df = filter_valid_events(df_original)
    
    # 통합 텍스트 코퍼스 생성
    text_corpus = (
        df['content_clean'].fillna('') + ' ' +
        df['place_clean'].fillna('') + ' ' +
        df['location_clean'].fillna('')
    )
    
    print(f"\n📝 텍스트 코퍼스 통계:")
    print(f"• 평균 길이: {text_corpus.str.len().mean():.0f}자")
    print(f"• 최대 길이: {text_corpus.str.len().max():.0f}자")
    print(f"• 최소 길이: {text_corpus.str.len().min():.0f}자")
    
    # 샘플 데이터 확인
    print("\n🔍 필터링된 데이터 샘플:")
    display(df[['content', 'place', 'price_adv', 'loc_sigu']].head(3))
else:
    df = None

## 3. TF-IDF 모델 학습

In [ ]:
if df is not None:
    print("🔤 TF-IDF 모델 학습 시작...")
    
    # TF-IDF 벡터라이저 설정
    tfidf_vectorizer = TfidfVectorizer(
        max_features=10000,
        ngram_range=(1, 2),
        min_df=2,
        stop_words='english'
    )
    
    # 텍스트 벡터화
    X_text_tfidf = tfidf_vectorizer.fit_transform(text_corpus)
    print(f"✅ TF-IDF 벡터 생성: {X_text_tfidf.shape}")
    
    # 메타데이터와 결합
    X_meta = meta_preprocessor.transform(df)
    X_combined_tfidf = hstack([X_text_tfidf, X_meta]).tocsr()
    print(f"✅ 통합 특성 벡터: {X_combined_tfidf.shape}")
    
    # KNN 모델 학습
    knn_tfidf = NearestNeighbors(metric='cosine', n_neighbors=20, n_jobs=-1)
    knn_tfidf.fit(X_combined_tfidf)
    print("✅ TF-IDF KNN 모델 학습 완료!")
    
    # 상위 특성 분석
    feature_names = tfidf_vectorizer.get_feature_names_out()
    tfidf_scores = X_text_tfidf.mean(axis=0).A1
    top_features = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)[:10]
    
    print("\n🏆 TF-IDF 상위 10개 특성:")
    for i, (feature, score) in enumerate(top_features, 1):
        print(f"{i:2d}. {feature:<15} (점수: {score:.4f})")

## 4. LSA (잠재 의미 분석) 모델 학습

In [ ]:
if df is not None:
    print("🧮 LSA 모델 학습 시작...")
    
    # Count Vectorizer (LSA에서 더 효과적)
    count_vectorizer = CountVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        min_df=2,
        stop_words='english'
    )
    
    X_text_count = count_vectorizer.fit_transform(text_corpus)
    print(f"✅ Count 벡터 생성: {X_text_count.shape}")
    
    # SVD로 차원 축소
    svd = TruncatedSVD(n_components=100, random_state=42)
    X_text_lsa = svd.fit_transform(X_text_count)
    print(f"✅ LSA 차원 축소: {X_text_lsa.shape}")
    print(f"✅ 설명된 분산 비율: {svd.explained_variance_ratio_.sum():.3f}")
    
    # 메타데이터와 결합
    X_meta_dense = X_meta.toarray() if hasattr(X_meta, 'toarray') else X_meta
    X_combined_lsa = np.hstack([X_text_lsa, X_meta_dense])
    print(f"✅ 통합 LSA 특성 벡터: {X_combined_lsa.shape}")
    
    # KNN 모델 학습
    knn_lsa = NearestNeighbors(metric='cosine', n_neighbors=20, n_jobs=-1)
    knn_lsa.fit(X_combined_lsa)
    print("✅ LSA KNN 모델 학습 완료!")
    
    # LSA 성분 시각화
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, 101), svd.explained_variance_ratio_, 'b-', alpha=0.7)
    plt.title('LSA 성분별 설명 분산')
    plt.xlabel('성분 번호')
    plt.ylabel('설명 분산 비율')
    plt.grid(True)
    plt.show()

## 5. Word2Vec 대안 모델 학습

In [ ]:
if df is not None:
    print("🔤 Word2Vec 대안 모델 학습 시작...")
    print("(HashingVectorizer를 사용하여 Word2Vec과 유사한 효과 구현)")
    
    # HashingVectorizer로 Word2Vec 대안 구현
    word2vec_hasher = HashingVectorizer(
        n_features=1000,
        ngram_range=(1, 3),
        binary=False,
        norm='l2',
        lowercase=True,
        stop_words='english'
    )
    
    X_text_w2v = word2vec_hasher.fit_transform(text_corpus)
    print(f"✅ Word2Vec 대안 벡터 생성: {X_text_w2v.shape}")
    
    # 메타데이터와 결합
    X_combined_w2v = hstack([X_text_w2v, X_meta]).tocsr()
    print(f"✅ 통합 Word2Vec 특성 벡터: {X_combined_w2v.shape}")
    
    # KNN 모델 학습
    knn_w2v = NearestNeighbors(metric='cosine', n_neighbors=20, n_jobs=-1)
    knn_w2v.fit(X_combined_w2v)
    print("✅ Word2Vec 대안 KNN 모델 학습 완료!")
    
    # 해싱 벡터 통계
    hash_density = X_text_w2v.nnz / (X_text_w2v.shape[0] * X_text_w2v.shape[1])
    print(f"\n📊 HashingVectorizer 통계:")
    print(f"• 벡터 밀도: {hash_density:.4f}")
    print(f"• 평균 비영 원소 수: {X_text_w2v.nnz / X_text_w2v.shape[0]:.1f}")
    print(f"• 특성 차원: {X_text_w2v.shape[1]:,}개")
    
    # Word2Vec 대안의 장점 설명
    print("\n💡 Word2Vec 대안의 특징:")
    print("• HashingVectorizer는 단어의 해시값을 이용하여 고정 크기 벡터 생성")
    print("• 메모리 효율적이며 새로운 단어에도 대응 가능")
    print("• n-gram을 통해 단어 순서와 문맥 정보 일부 보존")

## 6. 하이브리드 모델 구축

In [ ]:
if df is not None:
    print("🔄 하이브리드 모델 구축...")
    
    # 모든 모델을 딕셔너리로 정리
    models = {
        'tfidf': {
            'vectorizer': tfidf_vectorizer,
            'knn': knn_tfidf,
            'feature_matrix': X_combined_tfidf,
            'description': 'TF-IDF 기반 키워드 매칭',
            'type': 'sparse'
        },
        'lsa': {
            'count_vectorizer': count_vectorizer,
            'svd': svd,
            'knn': knn_lsa,
            'feature_matrix': X_combined_lsa,
            'description': 'LSA 기반 잠재 의미 분석',
            'type': 'dense'
        },
        'word2vec': {
            'hasher': word2vec_hasher,
            'knn': knn_w2v,
            'feature_matrix': X_combined_w2v,
            'description': 'HashingVectorizer 기반 단어 임베딩 대안',
            'type': 'sparse'
        }
    }
    
    print("✅ 하이브리드 모델 준비 완료!")
    print("\n🎯 학습된 모델들:")
    for name, model in models.items():
        matrix_shape = model['feature_matrix'].shape
        print(f"• {name.upper()}: {model['description']}")
        print(f"  - 특성 행렬 크기: {matrix_shape}")
        print(f"  - 데이터 타입: {model['type']}")
        print()
    
    # 하이브리드 추천 함수 정의
    def hybrid_recommend(query, top_k=5, weights=None):
        """하이브리드 추천: 여러 모델의 결과를 가중 평균"""
        if weights is None:
            weights = {'tfidf': 0.4, 'lsa': 0.3, 'word2vec': 0.3}
        
        all_results = []
        
        for model_name, weight in weights.items():
            try:
                q_vec = encode_query_for_model(query, model_name)
                distances, indices = models[model_name]['knn'].kneighbors(q_vec, n_neighbors=top_k*2)
                
                for idx, dist in zip(indices[0], distances[0]):
                    similarity = (1 - dist) * weight
                    all_results.append((idx, similarity, model_name))
            except Exception as e:
                print(f"Error in {model_name}: {e}")
        
        # 결과 집계 및 정렬
        result_dict = {}
        for idx, sim, model in all_results:
            if idx not in result_dict:
                result_dict[idx] = {'total_similarity': 0, 'models': []}
            result_dict[idx]['total_similarity'] += sim
            result_dict[idx]['models'].append(model)
        
        # 상위 k개 선택
        sorted_results = sorted(result_dict.items(), 
                               key=lambda x: x[1]['total_similarity'], 
                               reverse=True)[:top_k]
        
        return sorted_results
    
    print("💡 하이브리드 모델 특징:")
    print("• TF-IDF: 정확한 키워드 매칭 (가중치 40%)")
    print("• LSA: 의미적 유사성 분석 (가중치 30%)")
    print("• Word2Vec 대안: 단어 임베딩 효과 (가중치 30%)")
    print("• 각 모델의 강점을 결합하여 더 다양하고 정확한 추천 제공")

## 7. 모델 성능 비교

In [ ]:
if df is not None:
    # 쿼리 인코딩 함수
    def encode_query_for_model(query, model_name):
        """쿼리를 특정 모델용으로 인코딩"""
        keywords = query.get('keywords', '')
        price_max = query.get('price_max', 25000)
        location = query.get('location', 'unknown')
        
        # 메타데이터 처리
        meta_df = pd.DataFrame([{
            'price_adv': price_max,
            'price_door': price_max,
            'loc_sigu': location
        }])
        meta_vec = meta_preprocessor.transform(meta_df)
        
        model = models[model_name]
        
        if model_name == 'tfidf':
            text_vec = model['vectorizer'].transform([keywords])
            return hstack([text_vec, meta_vec])
        elif model_name == 'lsa':
            count_vec = model['count_vectorizer'].transform([keywords])
            text_reduced = model['svd'].transform(count_vec)
            meta_vec_dense = meta_vec.toarray() if hasattr(meta_vec, 'toarray') else meta_vec
            return np.hstack([text_reduced, meta_vec_dense])
        elif model_name == 'word2vec':
            text_vec = model['hasher'].transform([keywords])
            return hstack([text_vec, meta_vec])
    
    # 테스트 쿼리 정의
    test_queries = [
        {"keywords": "재즈 콘서트", "price_max": 50000, "location": "강남구"},
        {"keywords": "클래식 음악회", "price_max": 30000, "location": "종로구"},
        {"keywords": "록 페스티벌", "price_max": 80000, "location": "마포구"},
        {"keywords": "팝 공연", "price_max": 40000, "location": "서초구"}
    ]
    
    print("📊 모델 성능 비교 테스트")
    print("=" * 60)
    
    # 각 모델의 성능 측정
    performance_results = {}
    
    for model_name in models.keys():
        print(f"\n🔍 {model_name.upper()} 모델 테스트:")
        model_results = []
        
        for i, query in enumerate(test_queries, 1):
            try:
                q_vec = encode_query_for_model(query, model_name)
                distances, indices = models[model_name]['knn'].kneighbors(q_vec, n_neighbors=5)
                similarities = 1 - distances[0]
                avg_similarity = similarities.mean()
                model_results.append(avg_similarity)
                print(f"  쿼리 {i}: '{query['keywords']}' → 평균 유사도: {avg_similarity:.3f}")
            except Exception as e:
                print(f"  쿼리 {i} 오류: {e}")
                model_results.append(0)
        
        performance_results[model_name] = model_results
    
    # 성능 요약
    print("\n📈 모델별 평균 성능:")
    print("-" * 40)
    for model_name, results in performance_results.items():
        avg_performance = np.mean(results)
        print(f"• {model_name.upper():<10}: {avg_performance:.3f}")
    
    # 성능 시각화
    if len(performance_results) > 0:
        plt.figure(figsize=(12, 6))
        
        # 쿼리별 성능 비교
        plt.subplot(1, 2, 1)
        x = np.arange(len(test_queries))
        width = 0.25
        
        for i, (model_name, results) in enumerate(performance_results.items()):
            plt.bar(x + i*width, results, width, label=model_name.upper(), alpha=0.8)
        
        plt.xlabel('테스트 쿼리')
        plt.ylabel('평균 코사인 유사도')
        plt.title('쿼리별 모델 성능 비교')
        plt.xticks(x + width, [f"Q{i+1}" for i in range(len(test_queries))])
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 모델별 평균 성능
        plt.subplot(1, 2, 2)
        model_names = list(performance_results.keys())
        avg_scores = [np.mean(performance_results[name]) for name in model_names]
        
        bars = plt.bar(model_names, avg_scores, color=['skyblue', 'lightgreen', 'orange'], alpha=0.8)
        plt.ylabel('평균 유사도')
        plt.title('모델별 전체 평균 성능')
        plt.ylim(0, max(avg_scores) * 1.1)
        
        # 막대 위에 수치 표시
        for bar, score in zip(bars, avg_scores):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{score:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()

## 8. 추천 결과 시연

In [ ]:
if df is not None:
    def get_recommendations(query, model_name, top_k=3):
        """특정 모델로 추천 결과 반환"""
        try:
            q_vec = encode_query_for_model(query, model_name)
            distances, indices = models[model_name]['knn'].kneighbors(q_vec, n_neighbors=top_k)
            
            results = df.iloc[indices[0]].copy()
            results['similarity'] = 1 - distances[0]
            
            return results[['content', 'place', 'price_adv', 'loc_sigu', 'similarity']]
        except Exception as e:
            print(f"추천 오류 ({model_name}): {e}")
            return pd.DataFrame()
    
    # 시연용 쿼리들
    demo_queries = [
        {
            "keywords": "재즈 콘서트 라이브",
            "price_max": 60000,
            "location": "강남구"
        },
        {
            "keywords": "클래식 오케스트라",
            "price_max": 50000,
            "location": "종로구"
        }
    ]
    
    for query_idx, demo_query in enumerate(demo_queries, 1):
        print(f"\n🎯 데모 쿼리 {query_idx}: {demo_query}")
        print("=" * 80)
        
        # 각 모델별 추천 결과 비교
        for model_name in models.keys():
            print(f"\n🤖 {model_name.upper()} 모델 추천 결과:")
            print("-" * 50)
            
            recommendations = get_recommendations(demo_query, model_name, top_k=3)
            
            if not recommendations.empty:
                for i, (idx, row) in enumerate(recommendations.iterrows(), 1):
                    print(f"{i}. 📍 {row['place'][:40]}... ({row['loc_sigu']})")
                    print(f"   💰 가격: {row['price_adv']:,.0f}원")
                    print(f"   📊 유사도: {row['similarity']:.3f}")
                    print(f"   📝 내용: {row['content'][:80]}...")
                    print()
            else:
                print("❌ 추천 결과를 가져올 수 없습니다.")
        
        # 하이브리드 추천 시연
        print(f"\n🔄 HYBRID 모델 추천 결과 (가중 평균):")
        print("-" * 50)
        try:
            hybrid_results = hybrid_recommend(demo_query, top_k=3)
            for i, (idx, result_info) in enumerate(hybrid_results, 1):
                event = df.iloc[idx]
                print(f"{i}. 📍 {event['place'][:40]}... ({event['loc_sigu']})")
                print(f"   💰 가격: {event['price_adv']:,.0f}원")
                print(f"   📊 종합 점수: {result_info['total_similarity']:.3f}")
                print(f"   🤝 참여 모델: {', '.join(result_info['models'])}")
                print(f"   📝 내용: {event['content'][:80]}...")
                print()
        except Exception as e:
            print(f"❌ 하이브리드 추천 오류: {e}")

## 🎉 학습 완료!

축하합니다! 다중 모델 이벤트 추천 시스템의 전체 학습 과정이 완료되었습니다.

### ✅ 구현된 기능들:

#### 1. 데이터 전처리 및 필터링
- 본문이 10자 이하인 이벤트 제외
- 장소, 가격 정보가 없는 이벤트 제외
- 텍스트 정규화 및 정제

#### 2. 학습된 모델들
- **TF-IDF 모델**: 키워드 기반 정확한 매칭
- **LSA 모델**: 잠재 의미 분석으로 의미적 유사성 고려  
- **Word2Vec 대안**: HashingVectorizer로 단어 임베딩 효과
- **하이브리드 모델**: 세 모델의 가중 평균으로 종합적 추천

#### 3. 성능 평가
- 다양한 쿼리로 모델별 성능 비교
- 시각화를 통한 성능 분석
- 코사인 유사도 기반 정량적 평가

#### 4. 실제 추천 시연
- 각 모델별 추천 결과 비교
- 하이브리드 모델의 종합적 추천
- 유사도 점수 및 참여 모델 표시

### 🚀 다음 단계:
- `python simple_model_demo.py` 실행하여 추가 테스트
- 웹 애플리케이션에서 실제 사용자 테스트
- 성능 모니터링 및 개선
- 새로운 데이터로 모델 업데이트
- 다른 임베딩 방법 실험 (예: BERT, Sentence-BERT)

### 💡 개선 아이디어:
- 사용자 피드백을 통한 모델 가중치 조정
- 시간대별, 계절별 추천 가중치 적용
- 사용자 선호도 학습 및 개인화
- 실시간 인기도 반영

🎵 **이제 백엔드 서버를 실행하여 웹에서 추천 시스템을 체험해보세요!** 🎵